In [ ]:
from google.colab import drive
drive.mount('/content/Drive')

Mounted at /content/Drive


In [ ]:
!pip install torch torchvision torchaudio
!pip install transformers
!pip install tqdm

In [ ]:
import json
import random
import warnings
import torch
import time
import argparse
import json
import os
import time
import copy
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional


from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from transformers import BertForSequenceClassification, BertTokenizer, BertModel, AdamW

In [ ]:
# List of relation types
keys = ['no_relation', 'per:title', 'org:top_members/employees',
        'org:country_of_headquarters', 'per:parents', 'per:age',
        'per:countries_of_residence', 'per:children', 'org:alternate_names',
        'per:charges', 'per:cities_of_residence', 'per:origin', 'org:founded_by',
        'per:employee_of', 'per:siblings', 'per:alternate_names', 'org:website',
        'per:religion', 'per:stateorprovince_of_death', 'org:parents',
        'org:subsidiaries', 'per:other_family', 'per:stateorprovinces_of_residence',
        'org:members', 'per:cause_of_death', 'org:member_of',
        'org:number_of_employees/members', 'per:country_of_birth',
        'org:shareholders', 'org:stateorprovince_of_headquarters',
        'per:city_of_death', 'per:date_of_birth', 'per:spouse',
        'org:city_of_headquarters', 'per:date_of_death', 'per:schools_attended',
        'org:political/religious_affiliation', 'per:country_of_death',
        'org:founded', 'per:stateorprovince_of_birth', 'per:city_of_birth',
        'org:dissolved']

# Assigning indices to the list elements and storing them in a dictionary
rel2id = {key: idx for idx, key in enumerate(keys)}
id2rel = {idx: key for key, idx in rel2id.items()}

# Printing the dictionaries
print(rel2id)
print(id2rel)

{'no_relation': 0, 'per:title': 1, 'org:top_members/employees': 2, 'org:country_of_headquarters': 3, 'per:parents': 4, 'per:age': 5, 'per:countries_of_residence': 6, 'per:children': 7, 'org:alternate_names': 8, 'per:charges': 9, 'per:cities_of_residence': 10, 'per:origin': 11, 'org:founded_by': 12, 'per:employee_of': 13, 'per:siblings': 14, 'per:alternate_names': 15, 'org:website': 16, 'per:religion': 17, 'per:stateorprovince_of_death': 18, 'org:parents': 19, 'org:subsidiaries': 20, 'per:other_family': 21, 'per:stateorprovinces_of_residence': 22, 'org:members': 23, 'per:cause_of_death': 24, 'org:member_of': 25, 'org:number_of_employees/members': 26, 'per:country_of_birth': 27, 'org:shareholders': 28, 'org:stateorprovince_of_headquarters': 29, 'per:city_of_death': 30, 'per:date_of_birth': 31, 'per:spouse': 32, 'org:city_of_headquarters': 33, 'per:date_of_death': 34, 'per:schools_attended': 35, 'org:political/religious_affiliation': 36, 'per:country_of_death': 37, 'org:founded': 38, 'per

In [ ]:
# tell pytorch to use the gpu if available
if torch.cuda.is_available():

    DEVICE = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not
else:
    print('No GPU available, using the CPU instead.')
    DEVICE = torch.device("cpu")

USE_CUDA = torch.cuda.is_available()
if USE_CUDA:
    print("using GPU")
else:
    print("using CPU")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-SXM2-16GB
using GPU


In [ ]:
#BERT tokenizer
model_directory = '/content/Drive/MyDrive/COMP61332_text_mining/RE/Tacred/Bert_RE/span-bert/spanbert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_directory)
# tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
print('BERT tokenizer loaded')

BERT tokenizer loaded


In [ ]:
def setup_seed(seed):
    # Sets the seed for generating random numbers for the CPU.

    # Sets the seed for generating random numbers for all GPUs.
    torch.cuda.manual_seed_all(seed)

    # Sets the seed for generating random numbers with NumPy.
    np.random.seed(seed)

    # Sets the seed for the built-in Python random module.
    random.seed(seed)

setup_seed(44)

In [ ]:
def load_tacred_dataset(file_path):
    """
    Load the TACRED dataset from a JSON file.

    Args:
    file_path (str): The path to the JSON file containing the dataset.

    Returns:
    dict: The loaded dataset.
    """
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

In [ ]:
def prepare_data(data):
    # Get the number of instances in the data
    n = len(data)

    # Initialize an empty list to store processed data
    info = []

    # Iterate through each instance in the data
    for i in range(n):
        # Initialize an empty dictionary for storing processed information of a single data instance
        single_data = {}

        # Extract start and end indices of subject and object entities
        ss = data[i]["subj_start"]
        se = data[i]["subj_end"]
        os = data[i]["obj_start"]
        oe = data[i]["obj_end"]

        # Extract subject and object tokens based on their start and end indices
        subj = data[i]['token'][ss: se+1]
        obj = data[i]['token'][os: oe+1]

        # Convert subject and object tokens into strings
        ent1 = ' '.join(subj)
        ent2 = ' '.join(obj)

        # Extract relation label
        rel = data[i]["relation"]

        # Concatenate all tokens to form the original sentence
        text = " ".join(data[i]["token"])

        # Store processed information in the dictionary
        single_data['rel'] = rel
        single_data['ent1'] = ent1
        single_data['ent2'] = ent2
        single_data['text'] = text

        # Append processed information of a single data instance to the list
        info.append(single_data)

    # Return the processed data
    return info

In [ ]:
def process_relation_extraction_data(info, max_length=64):
    # Initialize a dictionary to store processed data
    data = {}
    data['label'] = []  # List to store relation labels
    data['mask'] = []   # List to store attention masks
    data['text'] = []   # List to store tokenized and padded texts

    # Iterate through each instance in the provided 'info' data
    for line in info:
        # Check if the relation label is present in the 'rel2id' dictionary
        if line['rel'] not in rel2id:
            # If the relation label is not found, assign label 0 (for unknown relation)
            data['label'].append(0)
        else:
            # If the relation label is found, assign its corresponding index
            data['label'].append(rel2id[line['rel']])

        # Concatenate subject, object, and text to form a single sentence
        sent = line['ent1'] + line['ent2'] + line['text']

        # Tokenize the concatenated sentence and add special tokens
        indexed_tokens = tokenizer.encode(sent, add_special_tokens=True)

        # Determine the available length of the tokenized sentence
        avai_len = len(indexed_tokens)

        # Pad the tokenized sentence with 0s to match the maximum length
        while len(indexed_tokens) < max_length:
            indexed_tokens.append(0)

        # Trim the tokenized sentence to the maximum length
        indexed_tokens = indexed_tokens[:max_length]

        # Convert the tokenized sentence to a PyTorch tensor
        indexed_tokens = torch.tensor(indexed_tokens).long().unsqueeze(0)  # (1, L)

        # Create an attention mask for the tokenized sentence
        att_mask = torch.zeros(indexed_tokens.size()).long()  # (1, L)
        att_mask[0, :avai_len] = 1

        # Append the tokenized and padded sentence, and its attention mask to the data dictionary
        data['text'].append(indexed_tokens)
        data['mask'].append(att_mask)

    return data

In [ ]:
def convert_data_to_tensors(data):
    # Extract text, mask, and label from the data
    text = data['text']
    mask = data['mask']
    label = data['label']

    # Convert text and mask tensors to numpy arrays
    text = [t.numpy() for t in text]
    mask = [t.numpy() for t in mask]

    # Convert numpy arrays to PyTorch tensors
    text = torch.tensor(text)
    mask = torch.tensor(mask)
    label = torch.tensor(label)

    return text, mask, label

In [ ]:
class BERT_Classifier(nn.Module):
    def __init__(self, label_num):
        super().__init__()
        # Initialize the BERT encoder from pre-trained weights
        # self.encoder = BertModel.from_pretrained('bert-base-uncased')
        model_directory = '/content/Drive/MyDrive/COMP61332_text_mining/RE/Tacred/Bert_RE/span-bert/spanbert-base-cased'

        # Load the tokenizer and model
        self.encoder = BertModel.from_pretrained(model_directory)
        # Dropout layer to prevent overfitting
        self.dropout = nn.Dropout(0.1, inplace=False)
        # Fully connected layer for classification
        self.fc = nn.Linear(768, label_num)  # 768 is the hidden size of BERT
        # Cross-entropy loss criterion
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, x, attention_mask, label=None):
        # Pass the input through the BERT encoder
        x = self.encoder(x, attention_mask=attention_mask)[0]  # Output is tuple (last_hidden_state, pooler_output), we take the last_hidden_state
        # Take only the first token's output (CLS token)
        x = x[:, 0, :]
        # Apply dropout
        x = self.dropout(x)
        # Pass through the fully connected layer
        x = self.fc(x)
        # If label is not provided, return logits only
        if label is None:
            return None, x
        else:
            # Calculate the cross-entropy loss and return both loss and logits
            return self.criterion(x, label), x

labels_num=len(rel2id)
# print(labels_num)
model = BERT_Classifier(labels_num)
# Tell pytorch to run this model on the GPU.
model.cuda()

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertModel were not initialized from the model checkpoint at /content/Drive/MyDrive/COMP61332_text_mining/RE/Tacred/Bert_RE/span-bert/spanbert-base-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERT_Classifier(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
def train(net, train_dataset, dev_dataset, num_epochs, learning_rate, batch_size):

    print('Training...')

    # Set the network to training mode
    net.train()

    # Define the optimizer
    optimizer = optim.SGD(net.parameters(), lr=learning_rate)

    # Create a data loader for training data
    train_iter = torch.utils.data.DataLoader(train_dataset, batch_size, shuffle=True)

    for epoch in range(num_epochs):
        correct = 0
        total = 0
        iter = 0
        all_pred = []
        all_true = []

        # Initialize tqdm to show progress bar
        progress_bar = tqdm(train_iter, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch')

        for text, mask, y in progress_bar:
            iter += 1
            optimizer.zero_grad()

            # If the batch size is not equal to the specified batch size, break the loop
            if text.size(0) != batch_size:
                break

            # Reshape text and mask tensors
            text = text.reshape(batch_size, -1)
            mask = mask.reshape(batch_size, -1)

            # Move tensors to GPU if available
            if USE_CUDA:
                text = text.cuda()
                mask = mask.cuda()
                y = y.cuda()

            # Forward pass
            loss, logits = net(text, mask, y)

            # Backpropagation
            loss.backward()
            optimizer.step()

            # Compute accuracy
            _, predicted = torch.max(logits.data, 1)
            total += text.size(0)
            correct += predicted.data.eq(y.data).cpu().sum()

            # Collect predictions and true labels
            all_pred.extend(predicted.cpu().numpy())
            all_true.extend(y.cpu().numpy())

            # Update progress bar
            progress_bar.set_postfix({'loss': loss.item(), 'accuracy': correct.item() / total})

        # After the end of each epoch, compute metrics
        accuracy = correct.cpu().numpy().tolist()/total
        loss = loss.detach().cpu()

        # Compute F1 scores
        macro_f1 = f1_score(all_true, all_pred, average='macro')
        micro_f1 = f1_score(all_true, all_pred, average='micro')
        weighted_f1 = f1_score(all_true, all_pred, average='weighted')

        # Print metrics
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Loss: {loss.mean().numpy().tolist()}, Accuracy: {accuracy}")
        print(f"Macro F1: {macro_f1}, Micro F1: {micro_f1}, Weighted F1: {weighted_f1}")

        # print()

        # print("Validation...")

        # dev_acc, dev_macro_f1, dev_micro_f1, dev_weighted_f1, _, _ = eval(net, dev_dataset, batch_size)


In [ ]:

def eval(net, dataset, batch_size):
    # Set the network to evaluation mode
    net.eval()

    # Create an iterator for the evaluation dataset
    eval_iter = DataLoader(dataset, batch_size, shuffle=False)

    # Lists to store predictions and true labels
    all_pred = []
    all_true = []

    # Lists to store evaluation metrics
    acc_list = []
    macro_f1_list = []
    micro_f1_list = []
    weighted_f1_list = []
    precision_list = []  # List for precision scores
    recall_list = []     # List for recall scores

    with torch.no_grad():
        correct = 0  # Counter for correctly classified samples
        total = 0    # Counter for total samples
        # Progress bar for visualization during evaluation
        progress_bar = tqdm(eval_iter, desc='Evaluation', unit='batch')

        for text, mask, y in progress_bar:
            if text.size(0) != batch_size:
                break

            text = text.reshape(batch_size, -1)
            mask = mask.reshape(batch_size, -1)

            if USE_CUDA:
                text, mask, y = text.cuda(), mask.cuda(), y.cuda()

            outputs = net(text, mask)
            loss, logits = outputs if isinstance(outputs, tuple) else (None, outputs)

            _, predicted = torch.max(logits, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

            all_pred.extend(predicted.cpu().numpy())
            all_true.extend(y.cpu().numpy())

        # Calculate overall accuracy and F1 scores
        acc = correct / total
        macro_f1 = f1_score(all_true, all_pred, average='macro')
        micro_f1 = f1_score(all_true, all_pred, average='micro')
        weighted_f1 = f1_score(all_true, all_pred, average='weighted')

        # Calculate precision and recall
        precision_macro = precision_score(all_true, all_pred, average='macro')
        recall_macro = recall_score(all_true, all_pred, average='macro')
        precision_micro = precision_score(all_true, all_pred, average='micro')
        recall_micro = recall_score(all_true, all_pred, average='micro')
        precision_weighted = precision_score(all_true, all_pred, average='weighted')
        recall_weighted = recall_score(all_true, all_pred, average='weighted')

        # Append metrics to respective lists
        acc_list.append(acc)
        macro_f1_list.append(macro_f1)
        micro_f1_list.append(micro_f1)
        weighted_f1_list.append(weighted_f1)
        precision_list.append((precision_macro, precision_micro, precision_weighted))
        recall_list.append((recall_macro, recall_micro, recall_weighted))

        # Print evaluation results
        print(f"Eval Result: Acc: {acc:.4f}, Macro F1: {macro_f1:.4f}, Micro F1: {micro_f1:.4f}, Weighted F1: {weighted_f1:.4f}")
        print(f"Precision (Macro, Micro, Weighted): {precision_macro:.4f}, {precision_micro:.4f}, {precision_weighted:.4f}")
        print(f"Recall (Macro, Micro, Weighted): {recall_macro:.4f}, {recall_micro:.4f}, {recall_weighted:.4f}")

        # Return evaluation metrics
        return acc_list, macro_f1_list, micro_f1_list, weighted_f1_list, precision_list, recall_list


In [ ]:
# Load data from JSON file
train_data = load_tacred_dataset('/content/Drive/MyDrive/COMP61332_text_mining/RE/Tacred/json/train.json')
dev_data = load_tacred_dataset('/content/Drive/MyDrive/COMP61332_text_mining/RE/Tacred/json/dev.json')
test_data = load_tacred_dataset('/content/Drive/MyDrive/COMP61332_text_mining/RE/Tacred/json/test.json')

In [ ]:
train_info = prepare_data(train_data)
dev_info = prepare_data(dev_data)
test_info = prepare_data(test_data)

print(len(train_info))
print(len(dev_info))
print(len(test_info))
print(dev_info[:50])

68124
22631
15509
[{'rel': 'per:title', 'ent1': 'Douglas Flint', 'ent2': 'chairman', 'text': 'At the same time , Chief Financial Officer Douglas Flint will become chairman , succeeding Stephen Green who is leaving to take a government job .'}, {'rel': 'no_relation', 'ent1': 'Julius Baer', 'ent2': 'Jeffrey White', 'text': 'U.S. District Court Judge Jeffrey White in mid-February issued an injunction against Wikileaks after the Zurich-based Bank Julius Baer accused the site of posting sensitive account information stolen by a disgruntled former employee .'}, {'rel': 'per:city_of_death', 'ent1': 'Montcourt', 'ent2': 'PARIS', 'text': 'PARIS 2009-07-07 11:07:32 UTC French media earlier reported that Montcourt , ranked 119 , was found dead by his girlfriend in the stairwell of his Paris apartment .'}, {'rel': 'no_relation', 'ent1': 'Freedom Communications', 'ent2': 'current', 'text': 'The current holdings of Blackstone-operated funds include Universal Orlando , Cadbury Schweppes , Freedom Com

In [ ]:
train_data = process_relation_extraction_data(train_info, 128)
dev_data = process_relation_extraction_data(dev_info, 128)
test_data = process_relation_extraction_data(test_info, 128)

print(len(train_data['label']))
print(len(dev_data['label']))
print(len(test_data['label']))

68124
22631
15509


In [ ]:
# Preprocess train data
train_text, train_mask, train_label = convert_data_to_tensors(train_data)
print("--train data--")
print(train_text.shape)
print(train_mask.shape)
print(train_label.shape)

# Preprocess dev data
dev_text, dev_mask, dev_label = convert_data_to_tensors(dev_data)
print("--eval data--")
print(dev_text.shape)
print(dev_mask.shape)
print(dev_label.shape)

# Preprocess test data
test_text, test_mask, test_label = convert_data_to_tensors(test_data)
print("--test data--")
print(test_text.shape)
print(test_mask.shape)
print(test_label.shape)

<ipython-input-11-846317e3e88f>:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  text = torch.tensor(text)


--train data--
torch.Size([68124, 1, 128])
torch.Size([68124, 1, 128])
torch.Size([68124])
--eval data--
torch.Size([22631, 1, 128])
torch.Size([22631, 1, 128])
torch.Size([22631])
--test data--
torch.Size([15509, 1, 128])
torch.Size([15509, 1, 128])
torch.Size([15509])


In [ ]:
# convert data to tensor
train_dataset = torch.utils.data.TensorDataset(train_text,train_mask,train_label)
dev_dataset = torch.utils.data.TensorDataset(dev_text,dev_mask,dev_label)

In [ ]:
train(model, train_dataset, dev_dataset, 25, 0.02, 32)

Training...


Epoch 1/25: 100%|█████████▉| 2128/2129 [06:56<00:00,  5.10batch/s, loss=0.961, accuracy=0.816]


Epoch 1/25
Loss: 0.9607898592948914, Accuracy: 0.8162006578947368
Macro F1: 0.042991175794774406, Micro F1: 0.816200657894737, Weighted F1: 0.7504136402451654


Epoch 2/25: 100%|█████████▉| 2128/2129 [06:57<00:00,  5.10batch/s, loss=0.578, accuracy=0.849]


Epoch 2/25
Loss: 0.5780540704727173, Accuracy: 0.848875117481203
Macro F1: 0.1593736053689775, Micro F1: 0.848875117481203, Weighted F1: 0.8140075844145855


...

Epoch 23/25
Loss: 0.013353997841477394, Accuracy: 0.9778841635338346
Macro F1: 0.8891490004968452, Micro F1: 0.9778841635338346, Weighted F1: 0.9777774464751668


Epoch 24/25: 100%|█████████▉| 2128/2129 [06:57<00:00,  5.10batch/s, loss=0.187, accuracy=0.98]


Epoch 24/25
Loss: 0.18715804815292358, Accuracy: 0.980483435150376
Macro F1: 0.89962524133692, Micro F1: 0.980483435150376, Weighted F1: 0.9804073141505946


Epoch 25/25: 100%|█████████▉| 2128/2129 [06:57<00:00,  5.10batch/s, loss=0.0149, accuracy=0.982]


Epoch 25/25
Loss: 0.014941203407943249, Accuracy: 0.982157542293233
Macro F1: 0.9095659174183462, Micro F1: 0.982157542293233, Weighted F1: 0.9820750016021499


In [ ]:
torch.save(model, 'span-bert_best_model.pth')
print("entire Model saved successfully.")

entire Model saved successfully.


In [ ]:
import json

# grid parameter
batch_sizes = [32, 64, 128]
learning_rates = [0.02, 0.002, 0.0002]
num_epochs = [15, 20, 25]

results = []

# grid search
for batch_size in batch_sizes:
    for lr in learning_rates:
        for epoch in num_epochs:
            print(f"Training with batch size = {batch_size}, lr = {lr}, epoch = {epoch}")
            # set to initial state
            model = BERT_Classifier(labels_num)
            # Tell pytorch to run this model on the GPU.
            if torch.cuda.is_available():
                model.cuda()
            # run model
            train(model, train_dataset, dev_dataset, epoch, lr, batch_size)

            # evaluate the model
            test_dataset = torch.utils.data.TensorDataset(test_text, test_mask, test_label)
            acc_list, macro_f1_list, micro_f1_list, weighted_f1_list, precision_list, recall_list = eval(model, test_dataset, batch_size)

            # save result
            result = {
                'batch_size': batch_size,
                'lr': lr,
                'epoch': epoch,
                'acc_list': acc_list,
                'macro_f1_list': macro_f1_list,
                'micro_f1_list': micro_f1_list,
                'weighted_f1_list': weighted_f1_list,
                'precision_list': precision_list,
                'recall_list': recall_list
            }
            results.append(result)

            # save in the file
            with open('grid_search_results.txt', 'a') as file:
                file.write(json.dumps(result) + '\n')


for result in results:
    print(f"Batch size: {result['batch_size']}, LR: {result['lr']}, Epoch: {result['epoch']}")
    print(f"Accuracy List: {result['acc_list']}")
    print(f"Macro F1 List: {result['macro_f1_list']}")
    print(f"Micro F1 List: {result['micro_f1_list']}")
    print(f"Weighted F1 List: {result['weighted_f1_list']}")
    print(f"Precision List: {result['precision_list']}")
    print(f"Recall List: {result['recall_list']}")
    print("--------------------------------------------------")

In [ ]:
# load model 
model_path = "/content/Drive/MyDrive/COMP61332_text_mining/RE/Tacred/Bert_RE/inference/span-bert_best_model.pth"
model = torch.load(model_path)

In [ ]:

test_data = load_tacred_dataset('/content/Drive/MyDrive/COMP61332_text_mining/RE/Tacred/Bert_RE/experiments/test_rev.json')
test_info = prepare_data(test_data)
test_data = process_relation_extraction_data(test_info, 128)
test_text, test_mask, test_label = convert_data_to_tensors(test_data)
test_dataset = torch.utils.data.TensorDataset(test_text, test_mask, test_label)
acc_list, macro_f1_list, micro_f1_list, weighted_f1_list, precision_list, recall_list= eval(model, test_dataset, 32)

<ipython-input-11-846317e3e88f>:12: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  text = torch.tensor(text)
Evaluation: 100%|█████████▉| 484/485 [00:33<00:00, 14.40batch/s]
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Eval Result: Acc: 0.8910, Macro F1: 0.5591, Micro F1: 0.8910, Weighted F1: 0.8916
Precision (Macro, Micro, Weighted): 0.5917, 0.8910, 0.8970
Recall (Macro, Micro, Weighted): 0.5990, 0.8910, 0.8910


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
